# Exploratory Data Analysis

Here I try to load sas7bdat

In [8]:
# import your package
%load_ext autoreload
%autoreload 2
import src

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.

True

In [10]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [11]:
import numpy as np
import pandas as pd

from pathlib import Path
from tqdm.auto import tqdm
import logging, os

logging.basicConfig(stream=os.sys.stdout, level=logging.INFO)
logger = logging.getLogger('nb')
logger.setLevel(logging.INFO)

In [12]:
# Import the backtrader platform
import backtrader as bt

# Data

## Raw 2 interim

In [13]:
import subprocess

In [51]:
path_raw = Path("../data/raw/")
path_interim = Path("../data/interim/")
import functools
from src.data.helpers import unlz as unlz2
unlz = functools.partial(unlz2, path_raw=path_raw, path_interim=path_interim)

In [29]:
# stdopd*.lz - standardized option prices
f = Path("../data/raw/optm_lz/stdopd/stdopd1996.sas7bdat.lz")
f2=unlz(f)
reader = pd.read_sas(f2, chunksize=10000 )
df = next(iter(reader))
df

,secid,date,days,forward_price,strike_price,premium,impl_volatility,delta,gamma,theta,vega,cp_flag
0,5005.0,1996-01-04,10.0,8.575951,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,b'C'
1,5005.0,1996-01-04,10.0,8.575951,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,b'P'
2,5005.0,1996-01-04,30.0,8.602683,8.602683,0.619482,0.633389,0.536157,0.255432,-3.982769,0.975378,b'C'
3,5005.0,1996-01-04,30.0,8.602683,8.602683,0.481446,0.489084,-0.475899,0.335968,-2.686470,0.976138,b'P'
4,5005.0,1996-01-04,60.0,8.642224,8.642224,0.913731,0.661684,0.553332,0.172053,-2.977034,1.372703,b'C'
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,5028.0,1996-01-04,30.0,25.368497,25.368497,2.013952,0.695683,-0.462838,0.079537,-11.490786,2.872552,b'P'
9996,5028.0,1996-01-04,60.0,25.485100,25.485100,2.645985,0.649700,0.552372,0.059440,-8.635462,4.049277,b'C'
9997,5028.0,1996-01-04,60.0,25.485100,25.485100,2.874471,0.701632,-0.447510,0.055832,-7.962661,4.042400,b'P'
9998,5028.0,1996-01-04,91.0,25.603726,25.603726,3.265200,0.652006,0.564628,0.047875,-7.101207,4.964102,b'C'


In [27]:
# hvold*.lz - realized volatility for each optionable security on each day
f = Path("../data/raw/optm_lz/hvold/hvold1996.sas7bdat.lz")
f2=unlz(f)
reader = pd.read_sas(f2, chunksize=10000 )
df = next(iter(reader))
df

,secid,date,days,volatility
0,5005.0,1996-01-04,10.0,0.245508
1,5005.0,1996-01-04,14.0,0.245508
2,5005.0,1996-01-04,30.0,0.245508
3,5005.0,1996-01-04,60.0,0.245508
4,5005.0,1996-01-04,91.0,0.245508
...,...,...,...,...
9995,5028.0,1996-01-22,730.0,0.762264
9996,5028.0,1996-01-22,1825.0,0.762264
9997,5028.0,1996-01-23,10.0,0.535992
9998,5028.0,1996-01-23,14.0,0.487810


In [28]:
# The Security Price files (secprdYYYY) contain the price history for the security for the corresponding calendar year. 
f = Path("../data/raw/optm_lz/secprd/secprd1996.sas7bdat.lz")
f2=unlz(f)
reader = pd.read_sas(f2, chunksize=10000 )
df = next(iter(reader))
df

,secid,date,low,high,close,volume,return,cfadj,open,cfret,shrout
0,5001.0,1996-01-02,7.500,8.500,7.625,18900.0,NaN,1.0,8.500,1.000000,5052.0
1,5001.0,1996-01-03,7.500,8.500,8.500,5100.0,0.114754,1.0,7.500,1.000000,5052.0
2,5001.0,1996-01-04,8.500,8.500,8.500,100.0,0.000000,1.0,8.500,1.000000,5052.0
3,5001.0,1996-01-05,7.500,8.125,8.000,14800.0,-0.058824,1.0,8.000,1.000000,5052.0
4,5001.0,1996-01-08,8.000,8.000,8.000,200.0,0.000000,1.0,8.000,1.000000,5052.0
...,...,...,...,...,...,...,...,...,...,...,...
9995,5061.0,1996-09-13,25.125,25.625,25.375,288300.0,0.000000,1.0,25.500,1.020373,42717.0
9996,5061.0,1996-09-16,25.375,26.125,26.000,244800.0,0.024631,1.0,25.375,1.020373,42717.0
9997,5061.0,1996-09-17,25.750,26.750,26.625,331400.0,0.024038,1.0,26.125,1.020373,42717.0
9998,5061.0,1996-09-18,26.250,26.750,26.625,260600.0,0.000000,1.0,26.625,1.020373,42717.0


In [30]:
# vsurfd*.lz - interpolated volatility surface for each security on each day
f = Path("../data/raw/optm_lz/vsurfd/vsurfd1996.sas7bdat.lz")
f2=unlz(f)
reader = pd.read_sas(f2, chunksize=10000 )
df = next(iter(reader))
df

,secid,date,days,delta,impl_volatility,impl_strike,impl_premium,dispersion,cp_flag
0,5005.0,1996-01-04,10.0,-90.0,NaN,0.000000,0.000000,NaN,b'P'
1,5005.0,1996-01-04,10.0,-85.0,NaN,0.000000,0.000000,NaN,b'P'
2,5005.0,1996-01-04,10.0,-80.0,NaN,0.000000,0.000000,NaN,b'P'
3,5005.0,1996-01-04,10.0,-75.0,NaN,0.000000,0.000000,NaN,b'P'
4,5005.0,1996-01-04,10.0,-70.0,NaN,0.000000,0.000000,NaN,b'P'
...,...,...,...,...,...,...,...,...,...
9995,5005.0,1996-02-09,273.0,90.0,0.541948,5.082682,3.322793,0.000226,b'C'
9996,5005.0,1996-02-09,365.0,-90.0,0.528570,15.013450,7.063024,0.004415,b'P'
9997,5005.0,1996-02-09,365.0,-85.0,0.528509,14.124520,6.230890,0.001904,b'P'
9998,5005.0,1996-02-09,365.0,-80.0,0.528498,13.273070,5.457015,0.000823,b'P'


In [31]:
#  The Distribution file (distrd) contains information on a security's distributions and splits.
f = Path("../data/raw/optm_lz/distrd.sas7bdat.lz")
f2=unlz(f)
reader = pd.read_sas(f2, chunksize=10000 )
df = next(iter(reader))
df

,secid,record_date,seq_num,ex_date,amount,adj_factor,declare_date,payment_date,link_secid,distr_type,frequency,currency,approx_flag,cancel_flag,liquid_flag
0,5001.0,1995-03-15,1.0,1995-03-09,0.050,1.0,1995-02-17,1995-04-03,0.0,b'1',NaN,b'USD',b'0',b'0',b'0'
1,5001.0,1995-06-15,1.0,1995-06-13,0.050,1.0,1995-05-30,1995-07-03,0.0,b'1',NaN,b'USD',b'0',b'0',b'0'
2,5001.0,1995-09-15,1.0,1995-09-13,0.050,1.0,1995-08-30,1995-10-02,0.0,b'1',NaN,b'USD',b'0',b'0',b'0'
3,5001.0,1995-12-15,1.0,1995-12-18,0.050,1.0,1995-12-05,1996-01-02,0.0,b'1',NaN,b'USD',b'0',b'0',b'0'
4,5001.0,1996-02-07,1.0,1996-02-07,0.000,1.0,1996-01-24,1996-02-07,0.0,b'1',NaN,b'USD',b'0',b'1',b'0'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,6814.0,1997-06-13,1.0,1997-06-11,0.140,1.0,1997-05-28,1997-06-27,0.0,b'1',NaN,b'USD',b'0',b'0',b'0'
9996,6814.0,1997-08-06,1.0,1997-08-04,0.140,1.0,1997-07-23,1997-08-20,0.0,b'1',NaN,b'USD',b'0',b'0',b'0'
9997,6815.0,1995-11-30,1.0,1995-12-18,0.000,2.0,1995-11-08,1995-12-15,0.0,b'2',NaN,b'USD',b'0',b'0',b'0'
9998,6815.0,1997-06-25,1.0,1997-06-27,0.001,1.0,1997-06-13,1997-06-30,0.0,b'5',NaN,b'USD',b'0',b'0',b'0'


In [32]:
#  The Index Dividend file (idxdvd) contains the current dividend yield used for implied volatility calculations on index options.
f = Path("../data/raw/optm_lz/idxdvd.sas7bdat.lz")
f2=unlz(f)
reader = pd.read_sas(f2, chunksize=10000 )
df = next(iter(reader))
df

,secid,date,rate
0,100221.0,2015-10-20,4.551490
1,100221.0,2015-10-21,4.551840
2,100221.0,2015-10-22,4.549830
3,100221.0,2015-10-23,4.491418
4,100221.0,2015-10-26,4.493062
...,...,...,...
9995,100928.0,1996-11-04,1.977968
9996,100928.0,1996-11-05,2.018150
9997,100928.0,1996-11-06,2.127008
9998,100928.0,1996-11-07,2.245439


In [33]:
# presumably the names of all indexes
f = Path("../data/raw/optm_lz/indexd.sas7bdat.lz")
f2=unlz(f)
reader = pd.read_sas(f2, chunksize=10000 )
df = next(iter(reader))
df

,secid,ticker,cusip,exchange_d,issue_type,class,indexnam,issue,div_convention,exercise_style,am_set_flag
0,5613.0,b'LT88',b'59299Y93',32768.0,b'A',NaN,b'ML TRADEABLE USD SWAP INDEX',b'TR 2Y PAR CUPN',NaN,NaN,NaN
1,6100.0,b'MK87',b'85799X10',32768.0,b'A',NaN,b'S&P 400 DATA PROCESSING & OU',b'SUB IND TR',NaN,NaN,NaN
2,9530.0,b'ZZZZ',b'81699X93',32768.0,b'A',b'N',b'MAXIS NIKKEI 225 INDEX FUND',b'NAV',NaN,NaN,NaN
3,100000.0,b'AMEH',b'00000663',0.0,b'A',NaN,b'AMERICAN STOCK EXCHANGE',b'NUMBER OF NEW HIGHS',NaN,NaN,NaN
4,100001.0,b'AMEL',b'00000664',0.0,b'A',NaN,b'AMERICAN STOCK EXCHANGE',b'NUMBER OF NEW LOWS',NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
9995,125389.0,b'LMYG',b'26299Y41',32768.0,b'A',NaN,b'DOW JONES TOTAL STOCK MARKET',b'DJTSM US INVSERV',NaN,NaN,NaN
9996,125390.0,b'LMYH',b'26299Y42',32768.0,b'A',NaN,b'DOW JONES TOTAL STOCK MARKET',b'DJTSM US LEISGDS',NaN,NaN,NaN
9997,125391.0,b'LMYI',b'26299Y43',32768.0,b'A',NaN,b'DOW JONES TOTAL STOCK MARKET',b'DJTSM US LIFEINS',NaN,NaN,NaN
9998,125392.0,b'LMYJ',b'26299Y44',32768.0,b'A',NaN,b'DOW JONES TOTAL STOCK MARKET',b'DJTSM US MARTRNS',NaN,NaN,NaN


In [34]:
#  The Option Info file (opinfd) contains company information about underlying securities. 
f = Path("../data/raw/optm_lz/opinfd.sas7bdat.lz")
f2=unlz(f)
reader = pd.read_sas(f2, chunksize=10000)
df = next(iter(reader))
df

,secid,div_convention,exercise_style,am_set_flag
0,5005.0,NaN,b'A',b'0'
1,5015.0,NaN,b'A',b'0'
2,5016.0,NaN,b'A',b'0'
3,5022.0,NaN,b'A',b'0'
4,5028.0,NaN,b'A',b'0'
...,...,...,...,...
9995,213110.0,NaN,b'A',b'0'
9996,213114.0,NaN,b'A',b'0'
9997,213124.0,NaN,b'A',b'0'
9998,213126.0,NaN,b'A',b'0'


In [35]:
# presumably option names
f = Path("../data/raw/optm_lz/optionmnames.sas7bdat.lz")
f2=unlz(f)
reader = pd.read_sas(f2, chunksize=10000)
df = next(iter(reader))
df

,secid,symbol,optionid,root,suffix,effect_date,cusip,ticker,class,issuer,issue
0,5001.0,NaN,NaN,NaN,NaN,1996-01-02,b'00078110',b'ABSIE',NaN,b'ABS INDUSTRIES INC',NaN
1,5001.0,NaN,NaN,NaN,NaN,2007-03-08,b'00078110',b'ZZZZ',NaN,b'ABS INDS INC',b'COM'
2,5002.0,NaN,NaN,NaN,NaN,1996-01-01,b'00103010',b'AELNA',NaN,b'AEL INDUSTRIES - CI A',NaN
3,5003.0,NaN,NaN,NaN,NaN,1996-01-01,b'00103810',b'AFAP',NaN,b'AFA PROTECTIVE SYSTEMS INC',NaN
4,5003.0,NaN,NaN,NaN,NaN,1999-07-08,b'00103810',b'AFAPE',NaN,b'AFA PROTECTIVE SYSTEMS INC',NaN
...,...,...,...,...,...,...,...,...,...,...,...
9995,5097.0,b'0ADB7.3F',11384639.0,b'0ADB7',b'3F',1996-01-01,b'08172110',b'BNL',NaN,b'BENEFICIAL CORP',NaN
9996,5097.0,b'0ADC1.AB',11387307.0,b'0ADC1',b'AB',1996-01-01,b'08172110',b'BNL',NaN,b'BENEFICIAL CORP',NaN
9997,5097.0,b'0ADE9.E2',11397602.0,b'0ADE9',b'E2',1996-01-01,b'08172110',b'BNL',NaN,b'BENEFICIAL CORP',NaN
9998,5097.0,b'0ADF5.FB',11400699.0,b'0ADF5',b'FB',1996-01-01,b'08172110',b'BNL',NaN,b'BENEFICIAL CORP',NaN


In [41]:
# . The Option Volume file (opvold) contains daily total contract volume information for each underlying security. Volume is aggregated by calls, puts, and total.
f = Path("../data/raw/optm_lz/opvold.sas7bdat.lz")
f2=unlz(f)
reader = pd.read_sas(f2, chunksize=10000 )
df = next(iter(reader))
df

,secid,date,cp_flag,volume,open_interest
0,5005.0,1996-01-04,NaN,40.0,3747.0
1,5005.0,1996-01-04,b'C',40.0,2308.0
2,5005.0,1996-01-04,b'P',0.0,1439.0
3,5005.0,1996-01-05,NaN,5.0,3752.0
4,5005.0,1996-01-05,b'C',5.0,2313.0
...,...,...,...,...,...
9995,5033.0,1997-07-23,b'P',0.0,180.0
9996,5033.0,1997-07-24,NaN,0.0,518.0
9997,5033.0,1997-07-24,b'C',0.0,338.0
9998,5033.0,1997-07-24,b'P',0.0,180.0


In [42]:
# he Security Name file (secnmd) contains a historical record of changes to the ticker, issuer and issue descriptions, and CUSIP's for a security. 
f = Path("../data/raw/optm_lz/secnmd.sas7bdat.lz")
f2=unlz(f)
reader = pd.read_sas(f2, chunksize=10000 )
df = next(iter(reader))
df

,secid,effect_date,cusip,ticker,class,issuer,issue,sic
0,5001.0,1996-01-02,b'00078110',b'ABSIE',NaN,b'ABS INDUSTRIES INC',NaN,NaN
1,5001.0,2007-03-08,b'00078110',b'ZZZZ',NaN,b'ABS INDS INC',b'COM',b'3462'
2,5002.0,1996-01-01,b'00103010',b'AELNA',NaN,b'AEL INDUSTRIES - CI A',NaN,NaN
3,5003.0,1996-01-01,b'00103810',b'AFAP',NaN,b'AFA PROTECTIVE SYSTEMS INC',NaN,NaN
4,5003.0,1999-07-08,b'00103810',b'AFAPE',NaN,b'AFA PROTECTIVE SYSTEMS INC',NaN,NaN
...,...,...,...,...,...,...,...,...
9995,9504.0,2006-07-06,b'29410N10',b'ZZZZ',NaN,b'ENVISION DEV CORP',b'COM',b'5912'
9996,9505.0,1996-01-01,b'62626E10',b'?',b'?',b'MUNIHOLDINGS INSD FD IV INC',NaN,NaN
9997,9506.0,1996-01-01,b'48205010',b'?',b'?',b'JUPITER COMMUNICATIONS INC',NaN,NaN
9998,9507.0,1996-01-01,b'12694510',b'?',b'?',b'CVC INC',NaN,NaN


In [46]:
# The Security file (securd) contains identifying information for all equity and index securities known to OptionMetrics. 
f = Path("../data/raw/optm_lz/securd.sas7bdat.lz")
f2=unlz(f)
reader = pd.read_sas(f2, chunksize=10000 )
df = next(iter(reader))
df

,secid,cusip,ticker,sic,index_flag,exchange_d,class,issue_type,industry_group
0,5001.0,b'00078110',b'ZZZZ',b'3462',b'0',0.0,NaN,b'0',NaN
1,5002.0,b'00103010',b'AELNA',NaN,b'0',0.0,NaN,NaN,NaN
2,5003.0,b'00103810',b'AFAP',b'7382',b'0',16.0,NaN,b'0',NaN
3,5004.0,b'62544010',b'MTI',NaN,b'0',0.0,NaN,NaN,NaN
4,5005.0,b'00190710',b'ASTA',NaN,b'0',0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
9995,105579.0,b'41618720',b'ZZZZ',b'7389',b'0',0.0,NaN,b'0',850.0
9996,105580.0,b'41619620',b'HHS',b'8742',b'0',1.0,NaN,b'0',721.0
9997,105581.0,b'41651510',b'HIG',b'6399',b'0',1.0,NaN,b'0',432.0
9998,105582.0,b'41711910',b'ZZZZ',b'2311',b'0',0.0,NaN,b'0',320.0


In [47]:
#  The Security file (securd) contains identifying information for all equity and index securities known to OptionMetrics. 
# extra col: issuer
f = Path("../data/raw/optm_lz/securd1.sas7bdat.lz")
f2=unlz(f)
reader = pd.read_sas(f2, chunksize=10000 )
df = next(iter(reader))
df

,secid,cusip,ticker,sic,index_flag,exchange_d,class,issue_type,industry_group,issuer
0,5001.0,b'00078110',b'ZZZZ',b'3462',b'0',0.0,NaN,b'0',NaN,b'ABS INDS INC'
1,5002.0,b'00103010',b'AELNA',NaN,b'0',0.0,NaN,NaN,NaN,b'AEL INDUSTRIES - CI A'
2,5003.0,b'00103810',b'AFAP',b'7382',b'0',16.0,NaN,b'0',NaN,b'AFA PROT SYS INC'
3,5004.0,b'62544010',b'MTI',NaN,b'0',0.0,NaN,NaN,NaN,b'MULTIGRAPHICS INC'
4,5005.0,b'00190710',b'ASTA',NaN,b'0',0.0,NaN,NaN,NaN,b'AST RESEARCH INC'
...,...,...,...,...,...,...,...,...,...,...
9995,105579.0,b'41618720',b'ZZZZ',b'7389',b'0',0.0,NaN,b'0',850.0,b'HARTCOURT COS INC'
9996,105580.0,b'41619620',b'HHS',b'8742',b'0',1.0,NaN,b'0',721.0,b'HARTE-HANKS INC'
9997,105581.0,b'41651510',b'HIG',b'6399',b'0',1.0,NaN,b'0',432.0,b'HARTFORD FINL SVCS GROUP INC'
9998,105582.0,b'41711910',b'ZZZZ',b'2311',b'0',0.0,NaN,b'0',320.0,b'HARTMARX CORP'


In [45]:
# The Zero Curve file (zerocd) contains the current zero-coupon interest rate curve used by OptionMetrics, 
f = Path("../data/raw/optm_lz/zerocd.sas7bdat.lz")
f2=unlz(f)
reader = pd.read_sas(f2, chunksize=10000 )
df = next(iter(reader))
df

,date,days,rate
0,1996-01-02,9.0,5.763067
1,1996-01-02,15.0,5.745902
2,1996-01-02,50.0,5.673317
3,1996-01-02,78.0,5.608884
4,1996-01-02,169.0,5.473762
...,...,...,...
9995,1996-11-27,3311.0,6.529681
9996,1996-11-27,3395.0,6.551927
9997,1996-11-27,3493.0,6.576233
9998,1996-11-27,3584.0,6.599010
